In [ ]:
import json
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # just keep tagger for lemmatization


## Dataset creation from visual genome files

In [ ]:
#relationships_json = pd.read_csv('relationships_train_json.json')

file = open('relationships.json')
relationships_json = json.load(file)

In [ ]:
len(relationships_json)

108077

In [ ]:
#relationships_json_reduced = relationships_json[0:20000]
relationships_json_reduced = relationships_json

In [ ]:
# image data to get image dimensions

#image_data_json =  pd.read_csv('images_train_json.json')
file.close()

# image data to get image dimensions

file = open('image_data.json')
image_data_json = json.load(file)
file.close()

In [ ]:
len(image_data_json)

108077

In [ ]:
#image_data_json

[{'coco_id': None,
  'flickr_id': None,
  'height': 600,
  'image_id': 1,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K_2/1.jpg',
  'width': 800},
 {'coco_id': None,
  'flickr_id': None,
  'height': 600,
  'image_id': 2,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/2.jpg',
  'width': 800},
 {'coco_id': None,
  'flickr_id': None,
  'height': 480,
  'image_id': 3,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/3.jpg',
  'width': 640},
 {'coco_id': None,
  'flickr_id': None,
  'height': 480,
  'image_id': 4,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/4.jpg',
  'width': 640},
 {'coco_id': None,
  'flickr_id': None,
  'height': 600,
  'image_id': 5,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/5.jpg',
  'width': 800},
 {'coco_id': None,
  'flickr_id': None,
  'height': 600,
  'image_id': 6,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/6.jpg',
  'width': 800},
 {'coco_id': None,
  'flickr_id': None,
  'height': 600,
  'image_id

In [ ]:
# NLP parser
def base_form(input_token):
  return " ".join([token.lemma_ for token in nlp(input_token.lower())])


In [ ]:
# Create a dataframe with combined data required for features and output

X = pd.DataFrame(columns = ['Image Height', 'Image_Width', 
                            'subject_name', 'subject_x', 'subject_y', 'subject_width', 'subject_height', 
                            'object_name', 'object_x', 'object_y', 'object_width', 'object_height',
                            'Image_ID'])
y = pd.DataFrame()

#subset of first 50k images
start = 0
end = 50000
for i in range(start, end):
  image_rel = relationships_json_reduced[i]

  if(image_data_json[i]['image_id'] != image_rel['image_id']):
    print("Image IDs don't match", image_data_json[i]['image_id'], image_rel['image_id'])
    break

  image_width = image_data_json[i]['width']
  image_height = image_data_json[i]['height']90

  for relationships_in_image in image_rel['relationships']:

    new_row = {'Image Height': image_height,
               'Image_Width': image_width,
               'subject_name': base_form(relationships_in_image['subject']['name'] if 'name' in relationships_in_image['subject'] else relationships_in_image['subject']['names'][0]),
               'subject_x': relationships_in_image['subject']['x'] + (relationships_in_image['subject']['w'] / 2),
               'subject_y': relationships_in_image['subject']['y'] + (relationships_in_image['subject']['h'] / 2), 
               'subject_width': relationships_in_image['subject']['w'],  
               'subject_height': relationships_in_image['subject']['h'], 
               'object_name': base_form(relationships_in_image['object']['name'] if 'name' in relationships_in_image['object'] else relationships_in_image['object']['names'][0]),
               'object_x': relationships_in_image['object']['x'] + (relationships_in_image['object']['w'] / 2),
               'object_y': relationships_in_image['object']['y'] + (relationships_in_image['object']['h'] / 2),
               'object_width': relationships_in_image['object']['w'],  
               'object_height': relationships_in_image['object']['h'],
               'Image_ID': image_data_json[i]['image_id']
               
               }

    X = X.append(new_row, ignore_index=True)

    y = y.append({'relationship': base_form(relationships_in_image['predicate'])}, ignore_index=True)


  if(i % 2000 == 0):
    print(i)
    X.to_csv('X_train_40-50k.csv')
    y.to_csv('y_train_40-50k.csv')


In [ ]:
X

,Image Height,Image_Width,subject_name,subject_x,subject_y,subject_width,subject_height,object_name,object_x,object_y,object_width,object_height,Image_ID
0,600,800,shade,256.0,434.0,274,192,sidewalk,439.0,453.0,722,290,1.0
1,600,800,man,268.0,380.0,60,262,shoe,412.0,499.0,48,28,1.0
2,600,800,car,516.0,364.0,74,98,headlight,525.5,373.5,23,15,1.0
3,600,800,sign,162.0,104.0,88,182,building,110.0,270.0,218,536,1.0
4,600,800,tree trunk,665.5,397.5,87,327,sidewalk,438.0,464.0,722,266,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,600,800,board,195.5,215.0,101,142,wall,401.0,267.0,796,530,10.0
419,600,800,ostirch,591.5,153.0,25,62,bookshelf,510.0,330.0,174,358,10.0
420,600,800,corkboard,198.5,215.5,81,117,wall,401.0,267.0,796,530,10.0
421,600,800,outlet,249.5,566.0,65,36,floor,437.5,535.0,721,126,10.0
